# Diseño conceptual de pipeline ETL para un escenario empresarial

**Escenario. Una cadena de tiendas minoristas quiere integrar datos de:**
- Sistema de punto de venta (ventas por tienda, producto, hora)
- Sistema de inventario (stock por producto y ubicación)
- CRM (información de clientes y segmentación)
- Sitio web (comportamiento online de clientes)

## Tareas de diseño

**Identificar fuentes de datos:**
- Base de datos SQL del POS
- API REST del sistema de inventario
- Archivos CSV exportados del CRM
- Logs del servidor web (JSON)

**Diseñar esquema destino:**

```sql
CREATE TABLE ventas_consolidadas (
    id_venta INTEGER PRIMARY KEY,
    fecha_venta DATE,
    id_tienda INTEGER,
    id_cliente INTEGER,
    id_producto INTEGER,
    cantidad INTEGER,
    precio_unitario DECIMAL(10,2),
    total_venta DECIMAL(10,2),
    canal_venta VARCHAR(20), -- 'online', 'tienda'
    segmento_cliente VARCHAR(20)
);
```

## Planificar transformaciones

- **Convertir timestamps a fechas consistentes**. Es importante asegurar que todas las fuentes respeten la misma zona horaria y manejar correctamente cambios como horario de verano, para evitar desfases en los reportes.
- **Calcular totales de venta (cantidad × precio)**. Aunque puede parecer redundante almacenarlo (pues se puede calcular desde la consulta), se justifica cuando el sistema requiere acceder frecuentemente al total o cuando se busca mejorar el rendimiento en dashboards y reportes.
- **Enriquecer con información de cliente (segmento)**. Esto permite crear campañas de marketing más efectivas, segmentación dinámica y envío de ofertas personalizadas por email o notificaciones.
- **Normalizar códigos de producto entre sistemas**. Asegura la integridad referencial entre el id_producto del POS y el del sistema de inventario, evitando inconsistencias y registros huérfanos.
- **Detectar y manejar ventas duplicadas**. Esto evita diferencias con la información contable y previene errores en KPIs clave como ingresos totales y unidades vendidas.

La tabla **ventas_consolidadas** no deja constancia de:
- Devoluciones de clientes, para lo cual se necesitarían campos como num_factura, num_nota_credito y motivo_devolución. Esto permitiría revertir ventas correctamente y determinar si el producto puede reincorporarse al inventario.
- Rebajas temporales, como descuentos por porcentaje, promociones “3x2”, o “segundo a mitad de precio”. Si el precio no viene ajustado desde el POS, habría que modelarlas para reflejar correctamente el valor real de la venta.

## Definir frecuencia y estrategia

- **Ventas del POS: carga incremental cada hora**. Para esto, el POS debe soportar incremental mediante:
    - timestamps con deduplicación,
    - CDC (Change Data Capture),
    - triggers de auditoría en la base de datos,
    - APIs que permitan consultar solo los cambios.
- **Inventario: actualización completa diaria**.
- **CRM: actualización semanal**.
- **Web analytics: carga batch diaria**.

Llama la atención que la carga de ventas desde el POS sea horaria y la de ventas desde la web sea diaria. Esto produce reportes de ventas y KPIs desalineados, por lo que se recomienda alinear la frecuencia de carga del canal web con la del POS, idealmente a carga incremental horaria, salvo que exista una razón para conservar el procesamiento diario. Por ejemplo, si la empresa depende de las ventas físicas en tiempo casi real para decisiones de reposición, mientras que las ventas web no impactan procesos críticos.

## Comentario final

El diseño ETL propuesto constituye un buen punto de partida, perfectible según las necesidades reales del negocio, especialmente en aspectos como la frecuencia de carga de cada sistema, el manejo de devoluciones y promociones, y la definición de transformaciones que permitan mantener la consistencia entre las distintas fuentes de datos.